# Credit CARD

La table credit_card_balance contient l’historique mensuel des cartes de crédit. Nous avons agrégé les soldes, les montants dus, les limites de crédit et les ratios d’utilisation au niveau SK_ID_CURR. Les agrégations incluent les moyennes, maximums, minimums et fréquences d’utilisation. Un flag CC_NO_RECORDS identifie les clients sans carte de crédit. Ces indicateurs permettent de mesurer la discipline financière du client dans l’usage de ses cartes.


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

CWD = Path.cwd()
PROJECT_ROOT = CWD.parent.parent   # notebooks/01_data_preparation -> projet root

DATA_RAW       = PROJECT_ROOT / "data" / "raw"
DATA_CLEAN     = PROJECT_ROOT / "data" / "clean"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_CLEAN.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("DATA_RAW       =", DATA_RAW)
print("DATA_PROCESSED =", DATA_PROCESSED)

# ============================
# IMPORT
# ============================
CC_PATH = DATA_RAW / "credit_card_balance.csv"
cc = pd.read_csv(CC_PATH)
print("credit_card_balance:", cc.shape)
cc.head()
cc_clean=cc.copy()


DATA_RAW       = c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2\data\raw
DATA_PROCESSED = c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2\data\processed
credit_card_balance: (3840312, 23)


## Typage

In [2]:
cc_clean["SK_ID_PREV"] = cc_clean["SK_ID_PREV"].astype(int)
cc_clean["SK_ID_CURR"] = cc_clean["SK_ID_CURR"].astype(int)
cc_clean["MONTHS_BALANCE"] = cc_clean["MONTHS_BALANCE"].astype(int)

num_cols = [
    "AMT_BALANCE", "AMT_CREDIT_LIMIT_ACTUAL",
    "AMT_DRAWINGS_ATM_CURRENT", "AMT_DRAWINGS_CURRENT",
    "AMT_DRAWINGS_OTHER_CURRENT", "AMT_DRAWINGS_POS_CURRENT",
    "AMT_INST_MIN_REGULARITY", "AMT_PAYMENT_CURRENT",
    "AMT_PAYMENT_TOTAL_CURRENT", "AMT_RECEIVABLE_PRINCIPAL",
    "AMT_RECIVABLE", "AMT_TOTAL_RECEIVABLE",
    "CNT_DRAWINGS_ATM_CURRENT", "CNT_DRAWINGS_CURRENT",
    "CNT_DRAWINGS_OTHER_CURRENT", "CNT_DRAWINGS_POS_CURRENT",
    "CNT_INSTALMENT_MATURE_CUM",
    "SK_DPD", "SK_DPD_DEF"
]

for col in num_cols:
    cc_clean[col] = pd.to_numeric(cc_clean[col], errors="coerce")

## Nettoyage

In [3]:
cc_clean["NAME_CONTRACT_STATUS"] = (
    cc_clean["NAME_CONTRACT_STATUS"]
    .astype("string")
    .str.strip()
)

## FLAG NA

In [4]:
for col in num_cols:
    cc_clean[f"CC_{col}_NA"] = cc_clean[col].isna().astype(int)

cc_clean["CC_STATUS_NA"] = cc_clean["NAME_CONTRACT_STATUS"].isna().astype(int)

## Features comportementales

In [5]:

# Utilisation du crédit
cc_clean["CC_UTILIZATION"] = (
    cc_clean["AMT_BALANCE"] /
    cc_clean["AMT_CREDIT_LIMIT_ACTUAL"].replace(0, np.nan)
)

# Total des retraits
cc_clean["CC_DRAWINGS_TOTAL"] = (
    cc_clean["AMT_DRAWINGS_ATM_CURRENT"].fillna(0) +
    cc_clean["AMT_DRAWINGS_POS_CURRENT"].fillna(0) +
    cc_clean["AMT_DRAWINGS_OTHER_CURRENT"].fillna(0)
)

# Flags de retard
cc_clean["CC_LATE"] = (cc_clean["SK_DPD"] > 0).astype(int)
cc_clean["CC_SEVERE_LATE"] = (cc_clean["SK_DPD_DEF"] > 0).astype(int)


cc_clean.shape

(3840312, 47)

## Agrégations numériques

In [6]:

num_agg_cols = num_cols + ["CC_UTILIZATION", "CC_DRAWINGS_TOTAL"]

agg_funcs = ["mean", "min", "max", "sum", "std"]

cc_num = cc_clean.groupby("SK_ID_PREV")[num_agg_cols].agg(agg_funcs)

cc_num.columns = [
    "CC_" + c[0].upper() + "_" + c[1].upper()
    for c in cc_num.columns
]

## Agrégations count

In [7]:
cc_counts = cc_clean.groupby("SK_ID_PREV").agg(
    CC_COUNT_MONTHS=("MONTHS_BALANCE", "count"),
    CC_COUNT_LATE=("CC_LATE", "sum"),
    CC_COUNT_SEVERE_LATE=("CC_SEVERE_LATE", "sum")
)


## Ratios

In [8]:
cc_ratios = pd.DataFrame(index=cc_counts.index)

cc_ratios["CC_RATIO_LATE"] = (
    cc_counts["CC_COUNT_LATE"] / cc_counts["CC_COUNT_MONTHS"]
)

cc_ratios["CC_RATIO_SEVERE_LATE"] = (
    cc_counts["CC_COUNT_SEVERE_LATE"] / cc_counts["CC_COUNT_MONTHS"]
)

cc_ratios = cc_ratios.fillna(0)


## Aggrégations catégorielles

In [9]:
cc_status_dummies = pd.get_dummies(
    cc_clean["NAME_CONTRACT_STATUS"],
    prefix="CC_STATUS"
)

cc_status_dummies["SK_ID_PREV"] = cc_clean["SK_ID_PREV"]

cc_status = cc_status_dummies.groupby("SK_ID_PREV").sum()

for col in cc_status.columns:
    cc_status[col + "_RATIO"] = (
        cc_status[col] / cc_counts["CC_COUNT_MONTHS"]
    )

## Agrégations temporelles

In [10]:
cc_time = cc_clean.groupby("SK_ID_PREV").agg(
    CC_LAST_MONTH=("MONTHS_BALANCE", "max"),
    CC_FIRST_MONTH=("MONTHS_BALANCE", "min")
)

cc_time["CC_HISTORY_SPAN"] = (
    cc_time["CC_LAST_MONTH"] - cc_time["CC_FIRST_MONTH"]
)


## Fusion

In [11]:
cc_prev_agg = (
    cc_num
    .join(cc_counts)
    .join(cc_ratios)
    .join(cc_status)
    .join(cc_time)
)

## FLAG

In [12]:
cc_prev_agg["CC_NO_RECORDS"] = cc_prev_agg.isna().all(axis=1).astype(int)

cc_prev_agg = cc_prev_agg.reset_index()


print(cc_prev_agg.shape)
cc_prev_agg.head()



(104307, 129)


,SK_ID_PREV,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_SUM,CC_AMT_BALANCE_STD,CC_AMT_CREDIT_LIMIT_ACTUAL_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_MIN,CC_AMT_CREDIT_LIMIT_ACTUAL_MAX,CC_AMT_CREDIT_LIMIT_ACTUAL_SUM,CC_AMT_CREDIT_LIMIT_ACTUAL_STD,CC_AMT_DRAWINGS_ATM_CURRENT_MEAN,CC_AMT_DRAWINGS_ATM_CURRENT_MIN,CC_AMT_DRAWINGS_ATM_CURRENT_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,CC_AMT_DRAWINGS_ATM_CURRENT_STD,CC_AMT_DRAWINGS_CURRENT_MEAN,CC_AMT_DRAWINGS_CURRENT_MIN,CC_AMT_DRAWINGS_CURRENT_MAX,CC_AMT_DRAWINGS_CURRENT_SUM,CC_AMT_DRAWINGS_CURRENT_STD,CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN,CC_AMT_DRAWINGS_OTHER_CURRENT_MIN,CC_AMT_DRAWINGS_OTHER_CURRENT_MAX,CC_AMT_DRAWINGS_OTHER_CURRENT_SUM,CC_AMT_DRAWINGS_OTHER_CURRENT_STD,CC_AMT_DRAWINGS_POS_CURRENT_MEAN,CC_AMT_DRAWINGS_POS_CURRENT_MIN,CC_AMT_DRAWINGS_POS_CURRENT_MAX,CC_AMT_DRAWINGS_POS_CURRENT_SUM,CC_AMT_DRAWINGS_POS_CURRENT_STD,CC_AMT_INST_MIN_REGULARITY_MEAN,CC_AMT_INST_MIN_REGULARITY_MIN,CC_AMT_INST_MIN_REGULARITY_MAX,CC_AMT_INST_MIN_REGULARITY_SUM,CC_AMT_INST_MIN_REGULARITY_STD,CC_AMT_PAYMENT_CURRENT_MEAN,CC_AMT_PAYMENT_CURRENT_MIN,CC_AMT_PAYMENT_CURRENT_MAX,CC_AMT_PAYMENT_CURRENT_SUM,CC_AMT_PAYMENT_CURRENT_STD,CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_MIN,CC_AMT_PAYMENT_TOTAL_CURRENT_MAX,CC_AMT_PAYMENT_TOTAL_CURRENT_SUM,CC_AMT_PAYMENT_TOTAL_CURRENT_STD,CC_AMT_RECEIVABLE_PRINCIPAL_MEAN,CC_AMT_RECEIVABLE_PRINCIPAL_MIN,CC_AMT_RECEIVABLE_PRINCIPAL_MAX,CC_AMT_RECEIVABLE_PRINCIPAL_SUM,CC_AMT_RECEIVABLE_PRINCIPAL_STD,CC_AMT_RECIVABLE_MEAN,CC_AMT_RECIVABLE_MIN,CC_AMT_RECIVABLE_MAX,CC_AMT_RECIVABLE_SUM,CC_AMT_RECIVABLE_STD,CC_AMT_TOTAL_RECEIVABLE_MEAN,CC_AMT_TOTAL_RECEIVABLE_MIN,CC_AMT_TOTAL_RECEIVABLE_MAX,CC_AMT_TOTAL_RECEIVABLE_SUM,CC_AMT_TOTAL_RECEIVABLE_STD,CC_CNT_DRAWINGS_ATM_CURRENT_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_MIN,CC_CNT_DRAWINGS_ATM_CURRENT_MAX,CC_CNT_DRAWINGS_ATM_CURRENT_SUM,CC_CNT_DRAWINGS_ATM_CURRENT_STD,CC_CNT_DRAWINGS_CURRENT_MEAN,CC_CNT_DRAWINGS_CURRENT_MIN,CC_CNT_DRAWINGS_CURRENT_MAX,CC_CNT_DRAWINGS_CURRENT_SUM,CC_CNT_DRAWINGS_CURRENT_STD,CC_CNT_DRAWINGS_OTHER_CURRENT_MEAN,CC_CNT_DRAWINGS_OTHER_CURRENT_MIN,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX,CC_CNT_DRAWINGS_OTHER_CURRENT_SUM,CC_CNT_DRAWINGS_OTHER_CURRENT_STD,CC_CNT_DRAWINGS_POS_CURRENT_MEAN,CC_CNT_DRAWINGS_POS_CURRENT_MIN,CC_CNT_DRAWINGS_POS_CURRENT_MAX,CC_CNT_DRAWINGS_POS_CURRENT_SUM,CC_CNT_DRAWINGS_POS_CURRENT_STD,CC_CNT_INSTALMENT_MATURE_CUM_MEAN,CC_CNT_INSTALMENT_MATURE_CUM_MIN,CC_CNT_INSTALMENT_MATURE_CUM_MAX,CC_CNT_INSTALMENT_MATURE_CUM_SUM,CC_CNT_INSTALMENT_MATURE_CUM_STD,CC_SK_DPD_MEAN,CC_SK_DPD_MIN,CC_SK_DPD_MAX,CC_SK_DPD_SUM,CC_SK_DPD_STD,CC_SK_DPD_DEF_MEAN,CC_SK_DPD_DEF_MIN,CC_SK_DPD_DEF_MAX,CC_SK_DPD_DEF_SUM,CC_SK_DPD_DEF_STD,CC_CC_UTILIZATION_MEAN,CC_CC_UTILIZATION_MIN,CC_CC_UTILIZATION_MAX,CC_CC_UTILIZATION_SUM,CC_CC_UTILIZATION_STD,CC_CC_DRAWINGS_TOTAL_MEAN,CC_CC_DRAWINGS_TOTAL_MIN,CC_CC_DRAWINGS_TOTAL_MAX,CC_CC_DRAWINGS_TOTAL_SUM,CC_CC_DRAWINGS_TOTAL_STD,CC_COUNT_MONTHS,CC_COUNT_LATE,CC_COUNT_SEVERE_LATE,CC_RATIO_LATE,CC_RATIO_SEVERE_LATE,CC_STATUS_Active,CC_STATUS_Approved,CC_STATUS_Completed,CC_STATUS_Demand,CC_STATUS_Refused,CC_STATUS_Sent proposal,CC_STATUS_Signed,CC_STATUS_Active_RATIO,CC_STATUS_Approved_RATIO,CC_STATUS_Completed_RATIO,CC_STATUS_Demand_RATIO,CC_STATUS_Refused_RATIO,CC_STATUS_Sent proposal_RATIO,CC_STATUS_Signed_RATIO,CC_LAST_MONTH,CC_FIRST_MONTH,CC_HISTORY_SPAN,CC_NO_RECORDS
0,1000018,"74,946.2850","38,879.1450","136,695.4200","374,731.4250","46,691.9962","81,000.0000",45000,135000,405000,"49,295.0302","5,400.0000",0.0000,"13,500.0000","27,000.0000","7,394.2545","29,478.9960","2,032.5600","69,156.9450","147,394.9800","29,890.0129",0.0000,0.0000,0.0000,0.0000,0.0000,"24,078.9960","2,032.5600","55,656.9450","120,394.9800","23,123.7342","2,594.0880",0.0000,"6,206.6700","12,970.4400","2,243.0974","5,541.7500","3,190.6350","9,000.0000","27,708.7500","2,723.5040","5,541.7500","3,190.6350","9,000.0000","27,708.7500","2,723.5040","72,298.1970","37,542.6450",

## Export

In [13]:
CC_OUT_PATH = DATA_PROCESSED / "credit_card_balance_final.csv"
cc_prev_agg.to_csv(CC_OUT_PATH, index=False)

print("Export terminé :", CC_OUT_PATH)

Export terminé : c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2\data\processed\credit_card_balance_final.csv
